# LCA and design optimization of an electric drone

````{card}
Author
^^^
Félix POLLET <br>
[felix.pollet@isae-supaero.fr](felix.pollet@isae-supaero.fr)
````

The aim of this notebook is to study the LCA of transporting packages using a battery-powered drone, and to redesign the drone to reduce the environmental impacts. In particular, we'll be putting ourselves in the role of a drone designer seeking to improve the environmental impact of its products.

```{figure} assets/figures/livraison-drone-la-poste.jpg
---
width: 500px
name: directive-fig
align: left
---
January 2024 - La Poste opens its 3rd drone package delivery service in Vercors, France. <br> (*Credits: La Poste*)
```

## 1. Goal and scope

```{exercise} Goal & scope for drone package transportation
:label: goal

Propose a definition for the goal and scope.
```

```{solution} goal
:class: dropdown

**Goal**<br>
The objective of this study is to evaluate the environmental impacts associated with transporting a package using a battery-powered drone. For this purpose, a reference drone will be assessed, and design alternatives will be evaluated for comparison with the reference. In particular, the sensitivity to the size of the drone and the battery technology will be assessed to explore opportunities for reducing the impacts.

**Product & Functional Unit**<br>
The operation consists of delivering a 2 kg package from point A to point B at a distance of 15 kilometres and a speed of 30 km/h. For this purpose, the drone is required to have a flight endurance of at least 30 minutes. It is assumed that this mission will be carried out 5 times a day (only during working days) for 2 years, i.e. 2500 trips. For better understanding, the functional unit will be reported per unit of package delivered rather than the total number of packages transported.

**Boundaries**<br>
The study encompasses the production of the drone components and their transportation to the final assembly site. Also included is the electricity required to power the drone during the use phase. The components are treated as wastes at the end of their life, such that no recycling is considered. The processes and facilities required for the final assembly of the drone are excluded from the study, as well as the ground infrastructures and the ancillary systems (e.g., battery chargers) that support the flight operation.

**Methodological choices**<br>
The impact assessment relies on the EU Product Environmental Footprint (PEF) method v3.1 which covers 16 environmental impacts. Most of the data for the inventory (LCI) relies on the EcoInvent database.
```


### Python setup

The following lines enable to setup the LCA project and import the necessary ressources. Run them without any change.

In [ ]:
# Import libraries
import brightway2 as bw
import os 
import lca_algebraic as agb
from sympy import init_printing
import matplotlib.pyplot as plt
from assets.helpers import *
NETWORK_PATH = './assets/lca_activities.html'

# Logging settings
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# Pretty print for Sympy
init_printing()

# Set current project
bw.projects.set_current('LCA_course')
agb.resetParams()

# Import database for aircraft LCA model
USER_DB = 'Foreground Drone'
agb.import_db("assets/models/db_drone.bw2")

## 2. Introduction to the *Parametric* Life Cycle Assessment

Let's start by importing the activity that aggregates all the necessary inputs and outputs for transporting a package with a drone. The construction of this activity is beyond the scope of this course, but interested readers can have a look at [this notebook](supporting_data/drone_lca_model.ipynb) to see how it has been implemented.

In [ ]:
# Activity representing the lifecycle related to the transportation of a package with a drone
drone_delivery = agb.findActivity(
    name="package transport, drone",
    db_name=USER_DB
)

You can display what is known as a *process tree*, which is a hierarchical representation of the activities and flows required to fulfil the functional unit being assessed. The rectangles represent individual activities. The grey ones are aggregated activities provided directly by the EcoInvent database. The arrows illustrate the flows between each activity, i.e. the amount of a given activity required to complete the parent activity (at the head of the arrow).

In [ ]:
# Display process tree
graph_activities(USER_DB, drone_delivery, NETWORK_PATH)
from IPython.display import IFrame
IFrame(src=NETWORK_PATH, width="100%", height="500px")

:::{note}
The process tree could be further developed for each EcoInvent activity. However, as a drone designer, we have no direct influence on how these activities are achieved. For example, the industrial processes for manufacturing an electric motor are beyond our control. 

Therefore, it is common in LCA studies to distinguish the **background** activities, which are considered fixed, from the **foreground** activities, which can be controlled by the organisation. 

Here, the design of the drone, which includes the size of the components and the energy consumption of the vehicle, is under the influence of the designer. Only these foreground activities can be manipulated to improve the environmental performance of the product.
:::

:::{hint} Parameterized LCA
As shown in the process tree, the amount of activities (i.e. the flows) required to fulfil the functional unit can be described by **parameters** rather than by fixed values. It is therefore possible to evaluate a range of alternative scenarios (typically, different drones) by varying the values of the parameters.

A description of the LCA parameters used in this study is provided below. Take some time to understand each parameter and how they relate to the design of the drone or to a broader context of operation.
:::

In [ ]:
# Print description of parameters
print_parameters(USER_DB)

Before going any further, we define the LCIA methods that will be employed in the study:

In [ ]:
# List of impact methods to consider
impact_methods = agb.findMethods("", mainCat="EF v3.1")
impact_methods = [impact_methods[i] for i in [0, 1, 5, 8, 9, 10, 11, 12, 15, 18, 19, 20, 21, 22, 23, 24]]

# Display the selected methods
df = pd.DataFrame(impact_methods, columns=['Methods package', 'Impact category', 'Indicator (unit of measure)'])
with pd.option_context('display.max_colwidth', None):
    display(df)

## 3. LCA of the reference drone

In this section, the LCA of a reference drone that meets the performance requirements defined in the functional unit is carried out. The technical specifications of this drone are provided below.

| Drone specificiations      |        |
|-----------------------|--------|
| Number of propellers / arms     | 4      |
| Total mass (including a 2 kg package)           | 4.6 kg     |
| Mass of propellers    | 0.43 kg      |
| Mass of motors     | 0.52 kg      |
| Mass of battery   | 1.08 kg       |
| Battery technology     | Li-Ion battery with NMC chemistry (Nickel-Manganese-Cobalt). Lifetime expectancy = 500 charge-discharge cycles.  |
| Mass of structures | 0.57  kg     |
| Energy consumption per ride (30-minutes mission with a 2 kg package) | 180 Wh    |

```{exercise} Impact assessment of the reference drone
:label: parameters

Fill the parameter values and run the impact assessment. 

What are the main contributors to the environmental impacts?
```

In [ ]:
# Value of parameters for the reference drone - TO COMPLETE
parameters = {
    "elec_mix": "eu",  # "fr", "eu" or "us"
    "n_missions": 2500,
    "mission_energy": 0.180,    # [kWh]
    "mass_batteries": 1.08,     # [kg]
    "battery_type": "nmc",      # "nmc" or "lfp"
    "n_cycles_battery": 500,
    "mass_propellers": 0.43,    # [kg]
    "mass_motors": 0.52,        # [kg]
    "mass_structure": 0.57,     # [kg]         
}

In [ ]:
# Compute impacts
agb.compute_impacts(
    
    # Activity to assess
    drone_delivery, 
    
    # list of impacts to consider
    impact_methods, 

    # values of parameters to apply for calculation
    **parameters,

    # Get contributions by lifecycle phase
    axis="phase",
)

In [ ]:
# Detailed contributions
results_lca = agb.compute_impacts(
    drone_delivery, 
    impact_methods, 
    **parameters,
    axis="subphase",
)
ax = plot_lca(results_lca, relative=True)
ax.set_title('LCA results');

```{exercise} Sensitivity assessment
:label: parameters

Are the results sensitive to the electricity mix (France, Europe or US)? To the estimated battery lifetime?
```

:::{caution}
Be careful when modifying the value of some LCA parameters. Although some of them can be changed independently of the others (e.g. the electricity mix is independent of the drone design), others are highly correlated. 

For example, a drone with a heavier battery will use more energy to power the flight, so the `mass_batteries` and `mission_energy` parameters cannot be changed independently. The value of these parameters should only be set after a design analysis to ensure consistent results. This is explained in bit more detail in the next section.
:::


## 4. Exploration of alternative drone candidates: optimizing the design to reduce environmental impact

### 4.a) A bit of context
Designing a product, such as a drone, involves making many decisions. For instance, propellers come in a range of sizes and materials. Similarly, batteries are available with different chemistries and capacities. Of course, some choices can be discarded as they won't satisfy the requirements. For example, a small battery won't keep a drone in the air for long. Still, there exist a quasi-infinite number of alternative solutions. With so many options, how do we choose the best design? This is where design optimization comes into play. 

Design optimization aims to identify the most optimal design given a set of requirements (e.g., the drone should be able to lift a one kilogram package during 25 minutes). However, it is crucial to define what constitutes "optimal". Usually, the best design is defined as a balance between different factors, with economic considerations playing a pivotal role. For example, we might want a drone that's cheap to manufacture and/or to operate.

In this section, we'll explore different drone designs, each optimized based on different criteria. Specifically, we'll consider:

- The lightest drone that still meets the mission requirements

- The one that consumes the least energy for performing the mission (is it the lightest?)

- The one with the minimal environmental impact, such as mitigating climate change.

:::{hint} Design optimization in a nutshell
Design optimization is like finding the perfect recipe for a dish. You have a list of ingredients (design options) and you want to combine them in the best way to create the most delicious meal (optimal design).

In engineering, it's similar. You have various design choices, such as materials, dimensions, and configurations. Design optimization helps you figure out the ideal combination of these choices to meet your goals, whether it's maximizing performance, minimizing cost, or reducing environmental impact.

Using physical models and numerical algorithms, design optimization searches through all the possible combinations to find the one that best meets your criteria.
:::

:::{hint} Mathematical formulation
Mathematically, a design optimization problem can be expressed as follows:

```{math}
:label: mymath
\begin{align*}
&{\operatorname{minimize}}& & f(x) \\
&\operatorname{with\;respect\;to}
& &x \in X \subseteq R^n \\
&\operatorname{subject\;to}
& &h_i(x) = 0, \quad i = 1, \dots,m_1 \\ 
&&&g_j(x) \leq 0, \quad j = 1,\dots,m_2 \\
\end{align*}
```

The **design variables** $x$ represent the parameters that can be changed in the design, such as dimensions, materials, or configurations (e.g. 4, 6 or 8 propellers). Each variable has a range or set of possible values that it can take.

The **objective function** $f(x)$ represent what must be optimized, for example the total mass of the drone.

The **constraints** $g(x)$ and $h(x)$ reflect some physical limitations (e.g. the propellers should not overlap) or performance requirements (e.g. flight endurance), among others.
:::

:::{note} *Design* versus *Sizing*
The definition of a design optimization problem can be divided into three main sub-problems: material/technology selection, configuration selection and finally sizing, i.e. dimensioning of the components.

In this section, only the sizing sub-problem is addressed. The drone configuration is fixed to a quadcopter, i.e. with four propellers. The material for the structure and the propellers is set to a composite fabric. Finally, a Li-ion NMC battery technology is selected. Therefore, only the following dimensions of the components will be varied during the optimization:
- Propeller diameter
  
- Motor size
  
- Battery capacity
  
- Arms length
:::


### 4.b) The calculations

Do not worry, we are not asking you to create a design optimization solver from scratch. Instead, this notebook provides you with the `sizing_optimization` function, which will perform the optimization routine for you. All you have to do is define some input values such as the performance requirements, call the `sizing_optimization` function, and voila! You have an optimal drone design on which you can then perform an impact assessment.

Let's start by importing this magic function and set up some values for the design problem:

In [ ]:
# Import the functions that executes the sizing optimization
from assets.drone_sizing import sizing_optimization

### Performance requirements for the mission
performances = [
    2.0,  # [kg] mass of the package to lift
    30.0, # [min] flight duration
]

### Reference technology for the battery
technology = [
    200.0,   # [Wh/kg] energy density of the battery (Li-Ion NMC technology)
]

### Other specific LCA parameters
lca_param = {
    "battery_type": "nmc",
    "n_cycles_battery": 500,
    "elec_mix": "eu",
    "n_missions": 2500,
    "lca_model": drone_delivery
}

Now, we run the sizing optimization with a mass minimization objective, i.e. we want to obtain the lightest drone possible:

In [ ]:
# Set objective to minimize
objective = 'mass'

# Run optimization
drone_parameters, lca_parameters = sizing_optimization(performances, technology, lca_param, objective)  # this optimizes the design according to the specifications

# Print parameters of the optimal drone
drone_parameters

This is our lightest drone that is able to complete the package delivery mission. Let's calculate and visualize its environmental impacts:

In [ ]:
# Compute LCA corresponding to optimal design
results_lca_mass = agb.compute_impacts(
    drone_delivery, 
    impact_methods, 
    **lca_parameters.to_dict()['value'],
    axis="subphase",
)
plot_lca(results_lca_mass, relative=True)

```{exercise} Comparative study
:label: comparative

Re-run the optimization by changing the objective. The objective can be one of the following:

- `'energy'`: to find the most energy-efficient design

- `('EF v3.1', 'climate change', 'global warming potential (GWP100)')` to minimize the climate change

- Or any other environmental impact category (refer to the LCIA methods table at the beginning of this notebook)

Then, compare the different drone designs and their LCA results. Identify the burden shifts (if any) from one life cycle stage to another, and/or from one impact category to another.

```

In [ ]:
# Set objective to minimize
objective = ...

# Run optimization
drone_parameters, lca_parameters = sizing_optimization(performances, technology, lca_param, objective)  # this optimizes the design according to the specifications

# Compute LCA of corresponding design
results_lca_energy = agb.compute_impacts(
    drone_delivery, 
    impact_methods, 
    **lca_parameters.to_dict()['value'],
    axis="subphase",
)
drone_parameters

In [ ]:
# ... TO COMPLETE

In [ ]:
# Plot comparison of LCA results
plot_lca_comparison([results_lca_mass, results_lca_energy, results_lca_climate])

### 4.c) Multi-objective optimization

In most situations, it is not possible to find a design that minimizes all environmental impacts simultaneously, as the goals might be conflicting. It is therefore necessary to make a trade-off between the various objectives to allow choosing one preferable solution among the alternatives. This is the purpose of **multi-objective optimization**.

One way to achieve multi-objective optimization is to build an aggreagated objective which reflects the relative importance of the different objectives. This is achieved by assigning weights $\omega_i$ to each objective $f_i$:
```{math}
\begin{align}
f = \sum_i^n \omega_i f_i
\end{align}
```

Properly setting and interpreting weights is crucial to ensure that the resulting solution reflects the intended trade-offs and preferences. In this case study, the weights are set up to reflect the [normalization and weighting factors of the EF methods](https://eplca.jrc.ec.europa.eu/permalink/EF3_1/Normalisation_Weighting_Factors_EF_3.1.xlsx).

```{exercise} Multi-objective optimization
:label: multiobj

Run the multi-objective optimization and compare the results with those obtained previously.  

Given the potentially low fidelity of the physical models used in the design optimization, and the uncertainty on the environmental data supporting the LCA, be critical of the accuracy of the results.
```

In [ ]:
# Multi-objective
objective = {
    ('EF v3.1', 'climate change', 'global warming potential (GWP100)'): 0.2106/7.55E+03,  # divide by EF normalisation factor and multiply by EF weighting factor
    ('EF v3.1', 'acidification', 'accumulated exceedance (AE)'): 0.0620/5.56E+01,
    ('EF v3.1', 'ecotoxicity: freshwater', 'comparative toxic unit for ecosystems (CTUe)'): 0.0192/5.67E+04,
    ('EF v3.1', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels'): 0.0832/6.50E+04,
    ('EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)'): 0.0280/1.61,
    ('EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'): 0.0296/1.95E+01,
    ('EF v3.1', 'eutrophication: terrestrial', 'accumulated exceedance (AE)'): 0.0371/1.77E+02,
    ('EF v3.1', 'human toxicity: carcinogenic', 'comparative toxic unit for human (CTUh)'): 0.0213/1.73E-05,
    ('EF v3.1', 'human toxicity: non-carcinogenic', 'comparative toxic unit for human (CTUh)'): 0.0184/1.29E-04,
    ('EF v3.1', 'ionising radiation: human health', 'human exposure efficiency relative to u235'): 0.0501/4.22E+03,
    ('EF v3.1', 'land use', 'soil quality index'): 0.0794/8.19E+05,
    ('EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'): 0.0755/6.36E-02,
    ('EF v3.1', 'ozone depletion', 'ozone depletion potential (ODP)'): 0.0631/5.23E-02,
    ('EF v3.1', 'particulate matter formation', 'impact on human health'): 0.0896/5.95E-04,
    ('EF v3.1', 'photochemical oxidant formation: human health', 'tropospheric ozone concentration increase'): 0.0478/4.09E+01,
    ('EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)'): 0.0851/1.15E+04,
}

# Run design optimization
drone_parameters, lca_parameters = sizing_optimization(performances, technology, lca_param, objective)  # this optimizes the design according to the specifications

# Compute LCA
results_lca_multi = agb.compute_impacts(
    drone_delivery, 
    impact_methods, 
    **lca_parameters.to_dict()['value'],
    axis="phase",
)
drone_parameters

In [ ]:
# Plot comparison of LCA results
plot_lca_comparison([results_lca_climate, results_lca_multi])

## 5. Comparison of battery technologies

There are several battery technologies or chemistries that can be used in air mobility applications. The following table shows the specifications of two types of batteries.

| Battery technology | Energy density | Average lifetime (number of charge-discharge cycles) |
|------------------|--------------------|------------------|
| Li-Ion NMC (Nickel-Manganese-Cobalt cathode)       | 200 Wh/kg          | 500              |
| Li-Ion LFP (Lithium Ferro Phosphate cathode)     | 130 Wh/kg          | 2000             |


```{exercise} Light vs durable: who wins?
:label: battery

The table above shows that the NMC battery is lighter than its counterpart for the same amount of stored energy. As a result, a drone design using this type of battery will be lighter and consume less energy during flight. However, NMC batteries are less durable than LFPs and will need to be replaced (so produced) more often.

Is it better to use an NMC or an LFP battery to design a drone with minimal environmental impacts? Run the following cells and analyse the results. Identify the burden shifts from one life cycle stage to another and from one impact category to another.

```


In [ ]:
### Performance requirements for the mission
performances = [
    2.0,  # [kg] mass of payload (package) to lift
    30.0, # [min] flight duration
]

### Reference technology for the battery
technology = [
    130.0,   # [Wh/kg] rapport énergie/masse de la batterie
]

### Other specific LCA parameters
lca_param = {
    "elec_mix": "eu",
    "n_missions": 2500,
    "n_cycles_battery": 2000,
    "battery_type": "lfp",
    "lca_model": drone_delivery
}

In [ ]:
# Run optimization
drone_parameters, lca_parameters = sizing_optimization(performances, technology, lca_param, objective)  # this optimizes the design according to the specifications

# Compute LCA corresponding to optimal design
results_lca_multi_lfp = agb.compute_impacts(
    drone_delivery, 
    impact_methods, 
    **lca_parameters.to_dict()['value'],
    axis="phase",
)
drone_parameters

In [ ]:
# Plot comparison of LCA results
plot_lca_comparison([results_lca_multi, results_lca_multi_lfp])

## Optional exercise: uncertainty assessment

To wrap up our exploration of LCA, it's important to recognize the role of uncertainty in LCA results. Among other sources of uncertainty, the high variability in the inventory data can lead to a wide range of possible outcomes for each environmental impact category. In this final exercise, we'll use Monte Carlo simulation to quantify this uncertainty.

```{hint} Monte Carlo simulation
Monte Carlo simulation involves running numerous evluations of the LCA model with varying LCI data inputs, allowing us to generate a distribution of outcomes for each impact category. By visualizing these distributions, we gain a better understanding of the uncertainty surrounding our LCA results and can identify the range of potential environmental impacts.
```

```{caution} Sources of uncertainty
Uncertainty in LCA results isn't solely due to variability in the inventory data. Other significant factors include the limitations and inaccuracies in the models used for the LCIA, as well as, especially during early design phases, uncertainty about the actual parameters of the system being studied (such as component weights).
```

In [ ]:
# Run Monte Carlo
res = lca_monte_carlo(
    drone_delivery, # the model
    impact_methods, # impacts to assess 

    # Number of Monte Carlo runs (1000 runs will take 1-2 minutes to compute)
    n_runs=1000, 
    
    # Whether uncertainty on characterization factors (LCIA models) is taken into account or not
    cfs_uncertainty = False,

    # Parameters of the model
    **lca_parameters.to_dict()['value']
)

In [ ]:
# Display statistical results
res.describe()

In [ ]:
# Plot distributions
import seaborn as sns

for column in res.columns:
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(4,4))
    sns.kdeplot(res[column], fill=True, ax=ax_hist)
    sns.boxplot(res[column], ax=ax_box, orient="h")